In [20]:
import random
import os
import numpy as np
import torch
import torch.nn.functional as F
import json
import warnings
from dotenv import load_dotenv
from transformers import PreTrainedModel, AutoConfig, AutoModel, AutoTokenizer
from typing import List, Optional, Dict
from tqdm import tqdm

os.environ["TOKENIZERS_PARALLELISM"] = "false" # Turn off parallelism for tokenizers from Hugging Face
device = torch.device(f"cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
warnings.filterwarnings('ignore')

Using device: cpu


In [12]:
# Load environment variables
load_dotenv(override=True)

# General variables
seed = int(os.getenv("SEED"))
batch_size = int(os.getenv("BATCH_SIZE"))
save_every = int(os.getenv("SAVE_EVERY"))

# Embeddings variables
corpus_path = os.getenv("CORPUS_PATH")
max_length_encoder = int(os.getenv("MAX_LENGTH_ENCODER"))
normalize_embeddings = os.getenv("NORMALIZE_EMBEDDINGS") == "True"
lower_case = os.getenv("LOWER_CASE") == "True"
normalize_text = os.getenv("NORMALIZE_TEXT") == "True"
embeddings_dir = os.getenv("EMBEDDINGS_DIR")

In [21]:
# Utility functions
def set_seeds(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

def read_json(file_path: str):
    with open(file_path, "rb") as reader:
        data = json.load(reader)
    return data


def write_json(data, file_path: str):
    with open(file_path, "w") as writer:
        json.dump(data, writer)

In [22]:
# Run seeder before proceeding
set_seeds(seed)

In [15]:
class Encoder(PreTrainedModel):
    """
    A wrapper class for encoding text using pre-trained transformer models with specified pooling strategy.
    """
    def __init__(self, config: AutoConfig, pooling: str = "average"):
        super().__init__(config)
        self.config = config
        if not hasattr(self.config, "pooling"):
            self.config.pooling = pooling

        self.model = AutoModel.from_pretrained(
            config.name_or_path, config=self.config
        )


    def forward(
        self,
        input_ids: torch.Tensor,
        attention_mask: Optional[torch.Tensor] = None,
        token_type_ids: Optional[torch.Tensor] = None,
    ) -> torch.Tensor:
        return self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )
    
    def encode(
        self, 
        input_ids: torch.Tensor,
        attention_mask: Optional[torch.Tensor] = None,
        token_type_ids: Optional[torch.Tensor] = None,
        normalize: bool = False
    ) -> torch.Tensor:
        model_output = self.forward(
            input_ids, 
            attention_mask,
            token_type_ids,
        )
        last_hidden = model_output["last_hidden_state"]
        last_hidden = last_hidden.masked_fill(~attention_mask[..., None].bool(), 0.)

        if self.config.pooling == "average":
            emb = last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]
        elif self.config.pooling == "cls":
            emb = last_hidden[:, 0]

        if normalize:
            emb = F.normalize(emb, dim=-1)

        return emb

In [16]:
class Retriever:
    """
    A class for retrieving document embeddings using a specified encoder, using a bi-encoder approach.
    """
    def __init__(
        self,
        device: torch.device,
        tokenizer: AutoTokenizer,
        query_encoder: Encoder,
        doc_encoder: Optional[Encoder] = None,
        max_length: int = 512,
        add_special_tokens: bool = True,
        norm_query_emb: bool = False,
        norm_doc_emb: bool = False,
        lower_case: bool = False,
        do_normalize_text: bool = False,
    ):
        
        self.device = device
        self.query_encoder = query_encoder.to(device)
        self.doc_encoder = self.query_encoder if doc_encoder is None else doc_encoder.to(device)
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.add_special_tokens = add_special_tokens
        self.norm_query_emb = norm_query_emb
        self.norm_doc_emb = norm_doc_emb
        self.lower_case = lower_case
        self.do_normalize_text = do_normalize_text
    

    def encode_corpus(
        self, 
        corpus_info: List[Dict[str, str]], 
        batch_size: int, 
        embeddings_dir: str, 
        save_every: int = 500
    ) -> None:
        os.makedirs(embeddings_dir, exist_ok=True)
        
        all_embeddings = []
        num_steps = 0

        nbatch = (len(corpus_info) - 1) // batch_size + 1
        print(f"Total number of batches: {nbatch}")
        with torch.no_grad():
            for k in tqdm(range(nbatch)):
                start_idx = k * batch_size
                end_idx = min((k + 1) * batch_size, len(corpus_info))
                # print(f"Start index: {start_idx}, End index: {end_idx}")

                corpus = [
                    c["title"] + " " + c["text"] if len(c["title"]) > 0 else c["text"] 
                    for c in corpus_info[start_idx: end_idx]
                ]
                if self.do_normalize_text:
                    corpus = [normalize_text.normalize(c) for c in corpus]
                if self.lower_case:
                    corpus = [c.lower() for c in corpus]

                doc_inputs = self.tokenizer(
                    corpus,
                    max_length=self.max_length,
                    padding=True,
                    truncation=True,
                    add_special_tokens=self.add_special_tokens,
                    return_tensors="pt",
                ).to(self.device)

                emb = self.doc_encoder.encode(**doc_inputs, normalize=self.norm_doc_emb)
                all_embeddings.append(emb)

                num_steps += 1

                if num_steps == save_every or k == nbatch - 1:
                    embeddings = torch.cat(all_embeddings, dim=0)
                    file_index = end_idx - 1  # Index of the last passage embedded in the batch
                    file_path = os.path.join(
                        embeddings_dir, f'{file_index}_embeddings.npy'
                    )
                    np.save(file_path, embeddings.cpu().numpy())
                    print(f"Saved embeddings for {file_index} passages.")
                    num_steps = 0
                    all_embeddings = []

In [18]:
def initialize_retriever() -> Retriever:
    config = AutoConfig.from_pretrained("facebook/contriever")
    encoder = Encoder(config).eval()
    tokenizer = AutoTokenizer.from_pretrained("facebook/contriever")
    retriever = Retriever(
        device=device, tokenizer=tokenizer, 
        query_encoder=encoder, 
        max_length=max_length_encoder,
        norm_doc_emb=normalize_embeddings,
        lower_case=lower_case,
        do_normalize_text=normalize_text
    )

    return retriever

In [9]:
corpus = list(read_json(corpus_path).values())

In [19]:
retriever = initialize_retriever()

BertConfig {
  "_name_or_path": "facebook/contriever",
  "architectures": [
    "Contriever"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.40.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



In [ ]:
retriever.encode_corpus(
    corpus, 
    batch_size=batch_size, 
    embeddings_dir=embeddings_dir,
    save_every=save_every
)